In [62]:
!pip install requests
!pip install PyPDF2
!pip install nltk

In [63]:
import pandas as pd 
import numpy as np

import PyPDF2
import nltk

import requests
import json
import io
import os

In [64]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/miguelzanchettin/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

## Importa dados

Utilizando API do Banco Central

In [21]:

url = 'https://www.bcb.gov.br/api/servico/sitebcb/copomminutes/ultimas?quantidade=10000&filtro='

with requests.get(url) as r:
        
    assert r.status_code < 400, f'[{r.status_code}]{r.text}'

    conteudo = json.loads(r.content)['conteudo']


indice = pd.DataFrame(conteudo)
indice.head()

,DataReferencia,ImagemCapa,Titulo,Url,LinkPagina,EsconderDataReferencia
0,2024-03-20T03:00:00Z,/content/publications/PublishingImages/Capas/c...,"261st Meeting - March 19-20, 2024",/content/copom/copomminutes/MINUTES 261.pdf,/en/publications/copomminutes/20032024,True
1,2024-01-31T03:00:00Z,/content/publications/PublishingImages/Capas/c...,"260th Meeting - January 30-31, 2024",/content/copom/copomminutes/MINUTES 260.pdf,/en/publications/copomminutes/31012024,True
2,2023-12-13T03:00:00Z,/content/publications/PublishingImages/Capas/c...,"259th Meeting - December 12-13, 2023",/content/copom/copomminutes/MINUTES 259.pdf,/en/publications/copomminutes/13122023,True
3,2023-11-01T03:00:00Z,/content/publications/PublishingImages/Capas/c...,"258th Meeting - October 31 - November 1, 2023",/content/copom/copomminutes/MINUTES 258.pdf,/en/publications/copomminutes/01112023,True
4,2023-09-20T03:00:00Z,/content/publications/PublishingImages/Capas/c...,"257th Meeting - September 19-20, 2023",/content/copom/copomminutes/MINUTES 257.pdf,/en/publications/copomminutes/20092023,True


In [24]:

indice['UrlCorreto'] = 'https://www.bcb.gov.br' + indice.Url
indice['UrlCorreto'].head()

0    https://www.bcb.gov.br/content/copom/copomminu...
1    https://www.bcb.gov.br/content/copom/copomminu...
2    https://www.bcb.gov.br/content/copom/copomminu...
3    https://www.bcb.gov.br/content/copom/copomminu...
4    https://www.bcb.gov.br/content/copom/copomminu...
Name: UrlCorreto, dtype: object

In [51]:

def ler_pdf(urls: pd.Series) -> str:
    
    link = []
    pagina = []
    texto = []

    for url in urls:
        
        response = requests.get(url)

        try:    
            with io.BytesIO(response.content) as pdf:
                reader = PyPDF2.PdfReader(pdf)

                for num_page, page in enumerate(reader.pages):

                    link.append(url)
                    pagina.append(num_page)
                    texto.append(page.extract_text())
        
        except:
            pass

    df = pd.DataFrame([link, pagina, texto]).T
    df.columns = ['UrlCorreto', 'Pagina', 'Conteudo']
    
    return df

# Exemplo
ler_pdf(indice['UrlCorreto'][100:102])

,UrlCorreto,Pagina,Conteudo
0,https://www.bcb.gov.br/content/copom/copomminu...,0,\n \n \n \n \n \n 1 \n \nMinutes of the 1 61t...
1,https://www.bcb.gov.br/content/copom/copomminu...,1,\n \n \n \n \n \n 2 \n1. After recording 0 .1...
2,https://www.bcb.gov.br/content/copom/copomminu...,2,\n \n \n \n \n \n 3 \nincrease of 1.1% in May...
3,https://www.bcb.gov.br/content/copom/copomminu...,3,"\n \n \n \n \n \n 4 \nGDP. On its turn, forei..."
4,https://www.bcb.gov.br/content/copom/copomminu...,4,\n \n \n \n \n \n 5 \n \n14. Regarding fiscal...
5,https://www.bcb.gov.br/content/copom/copomminu...,5,\n \n \n \n \n \n 6 \ndamage to these variabl...
6,https://www.bcb.gov.br/content/copom/copomminu...,6,\n \n \n \n \n \n 7 \nincompatible with produ...
7,https://www.bcb.gov.br/content/copom/copomminu...,7,\n \n \n \n \n \n 8 \n \n35. The IPCA rose 0....
8,https://www.bcb.gov.br/content/copom/copomminu...,8,\n \n \n \n \n \n 9 \nof 4.48% in the year an...
9,https://www.bcb.gov.br/content/copom/copomminu...,9,\n \n \n \n \n \n 10 \n \n50. Construction in...


In [86]:
# Salva em pasta para nao precisar baixar novamente
fname = 'BaseAtasCopom.csv'

if not os.path.isfile(fname):
    df = ler_pdf(indice['UrlCorreto'])
    df = pd.merge(indice, df, on='UrlCorreto', how='left')
    df.to_csv(fname)

df = pd.read_csv(fname)
df.head()


,Unnamed: 0,DataReferencia,ImagemCapa,Titulo,Url,LinkPagina,EsconderDataReferencia,UrlCorreto,Pagina,Conteudo
0,0,2024-03-20T03:00:00Z,/content/publications/PublishingImages/Capas/c...,"261st Meeting - March 19-20, 2024",/content/copom/copomminutes/MINUTES 261.pdf,/en/publications/copomminutes/20032024,True,https://www.bcb.gov.br/content/copom/copomminu...,0,1 bcb.gov.br \n \n \n \n \n \n \n \n \n...
1,1,2024-03-20T03:00:00Z,/content/publications/PublishingImages/Capas/c...,"261st Meeting - March 19-20, 2024",/content/copom/copomminutes/MINUTES 261.pdf,/en/publications/copomminutes/20032024,True,https://www.bcb.gov.br/content/copom/copomminu...,1,261st Meeting \nCOPOM \nMarch 19- 20 \n \n2...
2,2,2024-03-20T03:00:00Z,/content/publications/PublishingImages/Capas/c...,"261st Meeting - March 19-20, 2024",/content/copom/copomminutes/MINUTES 261.pdf,/en/publications/copomminutes/20032024,True,https://www.bcb.gov.br/content/copom/copomminu...,2,261st Meeting \nCOPOM \nMarch 19- 20 \n \n3...
3,3,2024-03-20T03:00:00Z,/content/publications/PublishingImages/Capas/c...,"261st Meeting - March 19-20, 2024",/content/copom/copomminutes/MINUTES 261.pdf,/en/publications/copomminutes/20032024,True,https://www.bcb.gov.br/content/copom/copomminu...,3,261st Meeting \nCOPOM \nMarch 19- 20 \n \n4...
4,4,2024-03-20T03:00:00Z,/content/publications/PublishingImages/Capas/c...,"261st Meeting - March 19-20, 2024",/content/copom/copomminutes/MINUTES 261.pdf,/en/publications/copomminutes/20032024,True,https://www.bcb.gov.br/content/copom/copomminu...,4,261st Meeting \nCOPOM \nMarch 19- 20 \n \n5...


## Análise textual

In [107]:

def tratar_texto(texto: str) -> list:
    
    if type(texto) != str:
        return

    texto = nltk.tokenize.wordpunct_tokenize(texto)
    texto = nltk.pos_tag(texto)

    return texto

df_ok = df.copy()
df_ok['Tags'] = df_ok['Conteudo'].apply(tratar_texto)

df_ok = df_ok.explode('Tags')
df_ok[['word', 'POS']] = df_ok.Tags.apply(pd.Series)

df_ok = df_ok[['DataReferencia', 'Titulo', 'Pagina', 'word', 'POS']]

# Count
df_ok = df_ok.groupby(['DataReferencia', 
                       'Titulo', 
                       'Pagina', 
                       'word', 
                       'POS'
                       ]).size().reset_index(name='Count')


df_ok.head()

,DataReferencia,Titulo,Pagina,word,POS,Count
0,2000-01-10T02:00:00Z,42nd Copom minutes,0,(,(,6
1,2000-01-10T02:00:00Z,42nd Copom minutes,0,),),6
2,2000-01-10T02:00:00Z,42nd Copom minutes,0,",",",",1
3,2000-01-10T02:00:00Z,42nd Copom minutes,0,.,.,2
4,2000-01-10T02:00:00Z,42nd Copom minutes,0,10,CD,3
